<a href="https://colab.research.google.com/github/vaishnavipathak/garbage-classification-cnn/blob/main/cnn_assignment_partB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# from torchvision.transforms as transforms
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split, Dataset
import os

!pip install -q gdown

!gdown '1cqAJ3b_TIEqHnKQGLGjxCBKS3MUatGa6'
DATA_DIR = "/content/garbage_data/"
!unzip -q Garbage_classification.zip -d {DATA_DIR}

print("got the Dataset")


Downloading...
From (original): https://drive.google.com/uc?id=1cqAJ3b_TIEqHnKQGLGjxCBKS3MUatGa6
From (redirected): https://drive.google.com/uc?id=1cqAJ3b_TIEqHnKQGLGjxCBKS3MUatGa6&confirm=t&uuid=a2ee1a03-9ea9-44b6-8559-0378a4597aa7
To: /content/Garbage_classification.zip
100% 43.0M/43.0M [00:01<00:00, 32.9MB/s]
got the Dataset


In [ ]:
IMG_SIZE = 224
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

dataset_folder = os.path.join(DATA_DIR, os.listdir(DATA_DIR)[0])
full_dataset = datasets.ImageFolder(dataset_folder)

val_split = 0.20
test_split = 0.20
train_size = int((1.0 - val_split - test_split) * len(full_dataset))
val_size = int(val_split * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

print(f"\nSplitting {len(full_dataset)} images into:")
print(f"- Training: {train_size} (60%)")
print(f"- Validation: {val_size} (20%)")
print(f"- Test: {test_size} (20%)")

generator = torch.Generator().manual_seed(42)

# class to apply transformations to subset
class DatasetWithTransforms(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        image, label = self.subset[index]
        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.subset)


train_subset, val_subset, test_subset = random_split(full_dataset, [train_size, val_size, test_size], generator=generator)

train_dataset = DatasetWithTransforms(train_subset, transform=data_transforms['train'])
val_dataset = DatasetWithTransforms(val_subset, transform=data_transforms['val'])
test_dataset = DatasetWithTransforms(test_subset, transform=data_transforms['val'])

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print("\nDataLoaders with the correct 60/20/20 split are now ready.")
print("Train DataLoader length:", len(train_loader.dataset))
print("Val DataLoader length:", len(val_loader.dataset))
print("Test DataLoader length:", len(test_loader.dataset))


Splitting 2527 images into:
- Training: 1516 (60%)
- Validation: 505 (20%)
- Test: 506 (20%)

DataLoaders with the correct 60/20/20 split are now ready.
Train DataLoader length: 1516
Val DataLoader length: 505
Test DataLoader length: 506


In [ ]:
# weights='IMAGENET1K_V2' loads the model with its learned weights
model = models.resnet50(weights='IMAGENET1K_V2')

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 6)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Model loaded, frozen, and modified for fine tuning")
print("New final layer:", model.fc)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 104MB/s]


Model loaded, frozen, and modified for fine-tuning.
New final layer: Linear(in_features=2048, out_features=6, bias=True)
